In [14]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.metrics import precision_score, recall_score, accuracy_score
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm

class WikiArtDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = [d for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
        self.files = []

        for label, cls in enumerate(self.classes):
            class_dir = os.path.join(root_dir, cls)
            for file_name in os.listdir(class_dir):
                file_path = os.path.join(class_dir, file_name)
                if os.path.isfile(file_path):
                    self.files.append((file_path, label))

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path, label = self.files[idx]
        try:
            image = Image.open(file_path).convert("RGB")
        except Exception as e:
            raise RuntimeError(f"Error loading image {file_path}: {e}")
        if self.transform:
            image = self.transform(image)
        return image, label

class DeepArtClassifier(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(DeepArtClassifier, self).__init__()

        # First hidden layer
        self.hidden1 = nn.Linear(input_dim, 2048)
        self.bn1 = nn.BatchNorm1d(2048)

        # Second hidden layer
        self.hidden2 = nn.Linear(2048, 1536) 
        self.bn2 = nn.BatchNorm1d(1536)

        # Third hidden layer
        self.hidden3 = nn.Linear(1536, 1024)
        self.bn3 = nn.BatchNorm1d(1024)

        # Fourth hidden layer
        self.hidden4 = nn.Linear(1024, 768)
        self.bn4 = nn.BatchNorm1d(768)

        # Fifth hidden layer
        self.hidden5 = nn.Linear(768, 512)
        self.bn5 = nn.BatchNorm1d(512)

        # Sixth hidden layer
        self.hidden6 = nn.Linear(512, 384)
        self.bn6 = nn.BatchNorm1d(384)

        # Seventh hidden layer
        self.hidden7 = nn.Linear(384, 256)
        self.bn7 = nn.BatchNorm1d(256)

        # Output layer
        self.linear = nn.Linear(256, output_dim)

    def forward(self, x):
        # Apply layers with ReLU activation and batch normalization
        x = torch.relu(self.bn1(self.hidden1(x)))
        x = torch.relu(self.bn2(self.hidden2(x)))
        x = torch.relu(self.bn3(self.hidden3(x)))
        x = torch.relu(self.bn4(self.hidden4(x)))
        x = torch.relu(self.bn5(self.hidden5(x)))
        x = torch.relu(self.bn6(self.hidden6(x)))
        x = torch.relu(self.bn7(self.hidden7(x)))
        return self.linear(x)

if __name__ == "__main__":
    # Setup
    data_path = r"C:\Users\yozev\OneDrive\Desktop\artFiltered"

    # Increased image resolution to 224x224
    transform = transforms.Compose([
        transforms.Resize((224, 224)), 
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225]) 
    ])

    # Load dataset
    print("Loading dataset...")
    dataset = WikiArtDataset(root_dir=data_path, transform=transform)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

    # Adjusted batch size for larger images
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    input_dim = 224 * 224 * 3 
    output_dim = len(dataset.classes)
    device = "cuda" if torch.cuda.is_available() else "cpu"

    print(f"\nNetwork Architecture:")
    print(f"Input dimension: {input_dim}")
    print("Hidden layers:")
    print("  Layer 1: 2048 neurons + BatchNorm")
    print("  Layer 2: 1536 neurons + BatchNorm")
    print("  Layer 3: 1024 neurons + BatchNorm")
    print("  Layer 4: 768 neurons + BatchNorm")
    print("  Layer 5: 512 neurons + BatchNorm")
    print("  Layer 6: 384 neurons + BatchNorm")
    print("  Layer 7: 256 neurons + BatchNorm")
    print(f"Output dimension: {output_dim}")
    print(f"Running on: {device}\n")

    # Initialize model, loss, and optimizer
    model = DeepArtClassifier(input_dim, output_dim).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    epochs = 35
    train_losses = []

    print("Starting training...")
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        pbar = tqdm(enumerate(train_loader), total=len(train_loader),
                   desc=f'Epoch [{epoch+1}/{epochs}]')

        for batch_idx, (images, labels) in pbar:
            images = images.view(images.size(0), -1).to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            avg_loss = running_loss / (batch_idx + 1)

            # Update progress bar
            pbar.set_postfix({'loss': f'{avg_loss:.4f}'})

        train_losses.append(avg_loss)
        print(f'Epoch [{epoch+1}/{epochs}] Loss: {avg_loss:.4f}')

    # Plot training loss
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Training Loss')
    plt.title('Training Loss Over Time')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.savefig('training_loss_deep.png')
    plt.close()

    # Evaluation
    print("\nEvaluating model...")
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Testing"):
            images = images.view(images.size(0), -1).to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.numpy())

    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted')
    recall = recall_score(all_labels, all_preds, average='weighted')

    print('\nFinal Results:')
    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')

Loading dataset...

Network Architecture:
Input dimension: 150528
Hidden layers:
  Layer 1: 2048 neurons + BatchNorm
  Layer 2: 1536 neurons + BatchNorm
  Layer 3: 1024 neurons + BatchNorm
  Layer 4: 768 neurons + BatchNorm
  Layer 5: 512 neurons + BatchNorm
  Layer 6: 384 neurons + BatchNorm
  Layer 7: 256 neurons + BatchNorm
Output dimension: 13
Running on: cuda

Starting training...


Epoch [1/35]: 100%|██████████| 1619/1619 [04:31<00:00,  5.96it/s, loss=2.4200]


Epoch [1/35] Loss: 2.4200


Epoch [2/35]: 100%|██████████| 1619/1619 [04:27<00:00,  6.04it/s, loss=2.3430]


Epoch [2/35] Loss: 2.3430


...

Epoch [34/35] Loss: 0.5645


Epoch [35/35]: 100%|██████████| 1619/1619 [08:19<00:00,  3.24it/s, loss=0.5335]


Epoch [35/35] Loss: 0.5335

Evaluating model...


Testing: 100%|██████████| 405/405 [01:45<00:00,  3.85it/s]


Final Results:
Accuracy: 0.3096
Precision: 0.3224
Recall: 0.3096
